In [57]:
import pandas as pd
import numpy as np
import math
import imblearn
from sklearn import tree
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.naive_bayes import GaussianNB
#from sklearn.utils import resample
from imblearn.over_sampling import SMOTE,RandomOverSampler
import warnings
warnings.filterwarnings('ignore')

# Path of the file to read
traingData_file_path = 'dm_data/training.csv'
testingData_file_path = 'dm_data/testing.csv'
data = pd.read_csv(traingData_file_path)
testing_data = pd.read_csv(testingData_file_path)

y = data.Attrition
X = data.drop(['Attrition'], axis=1)

from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)# fit predictor and target variable
x_rus, y_rus = rus.fit_resample(X, y)

yesCnt = 0
noCnt = 0
for i in y:
    if i == "Yes":
        yesCnt += 1
    else:
        noCnt += 1
obj1 = {"yes":yesCnt,"no":noCnt}
print('original dataset shape:', obj1)

yesCnt = 0
noCnt = 0
for i in y_rus:
    if i == "Yes":
        yesCnt += 1
    else:
        noCnt += 1
obj2 = {"yes":yesCnt,"no":noCnt}
print('Resample dataset shape', obj2)



rus2 = RandomOverSampler(random_state=42)
x_rus2, y_rus2 = rus2.fit_resample(X, y)
yesCnt = 0
noCnt = 0
for i in y:
    if i == "Yes":
        yesCnt += 1
    else:
        noCnt += 1
obj3 = {"yes":yesCnt,"no":noCnt}
print('original dataset shape:', obj3)

yesCnt = 0
noCnt = 0
for i in y_rus2:
    if i == "Yes":
        yesCnt += 1
    else:
        noCnt += 1
obj4 = {"yes":yesCnt,"no":noCnt}
print('Resample dataset shape', obj4)


"""smote = SMOTE()

labeledData_x = X.apply(LabelEncoder().fit_transform)
labeledData_y = LabelEncoder().fit_transform(np.ravel(y))

# fit predictor and target variable
x_smote, y_smote = smote.fit_resample(labeledData_x, labeledData_y)

yesCnt = 0
noCnt = 0
for i in y:
    if i == "Yes":
        yesCnt += 1
    else:
        noCnt += 1
obj5 = {"yes":yesCnt,"no":noCnt}
print('original dataset shape:', obj5)

yesCnt = 0
noCnt = 0
for i in y_smote:
    if i == "Yes":
        yesCnt += 1
    else:
        noCnt += 1
obj6 = {"yes":yesCnt,"no":noCnt}
print('Resample dataset shape', obj6)"""



original dataset shape: {'yes': 190, 'no': 987}
Resample dataset shape {'yes': 190, 'no': 190}
original dataset shape: {'yes': 190, 'no': 987}
Resample dataset shape {'yes': 987, 'no': 987}


'smote = SMOTE()\n\nlabeledData_x = X.apply(LabelEncoder().fit_transform)\nlabeledData_y = LabelEncoder().fit_transform(np.ravel(y))\n\n# fit predictor and target variable\nx_smote, y_smote = smote.fit_resample(labeledData_x, labeledData_y)\n\nyesCnt = 0\nnoCnt = 0\nfor i in y:\n    if i == "Yes":\n        yesCnt += 1\n    else:\n        noCnt += 1\nobj5 = {"yes":yesCnt,"no":noCnt}\nprint(\'original dataset shape:\', obj5)\n\nyesCnt = 0\nnoCnt = 0\nfor i in y_smote:\n    if i == "Yes":\n        yesCnt += 1\n    else:\n        noCnt += 1\nobj6 = {"yes":yesCnt,"no":noCnt}\nprint(\'Resample dataset shape\', obj6)'

In [58]:
# 將原資料分割成驗證集與訓練集
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state = 1)

my_list=[]
for i in range(0,10): # random under sampling 10 times
    X_pl,y_pl=rus.fit_resample(train_X,train_y)
    # X_pl,y_pl=rus2.fit_resample(train_X,train_y)  # random over sampling 
    my_list.append((X_pl,y_pl)) #forming tuples from samples

X_pl=[]
Y_pl=[]
for num in range(0,len(my_list)): # Creating the dataframes for input/output
    X_pl.append(pd.DataFrame(my_list[num][0])) # 將每次抽樣出的 feature 資料組合起來
    Y_pl.append(pd.DataFrame(my_list[num][1])) # 將每次抽樣出的 class 資料組合起來

X_pl_=pd.concat(X_pl) #Concatenating the DataFrames(轉成 data frame)
Y_pl_=pd.concat(Y_pl)

In [59]:

# 先將原資料 copy 出來避免後面轉換欄位型態時去更動到原資料

label_y_train = Y_pl_.copy()
label_y_test = test_y.copy()

label_x_train = X_pl_.copy()
label_x_test = test_X.copy()

# 轉換要被預測的 class 為 numerical  
label_encoder1 = LabelEncoder()
label_encoder1.fit(Y_pl_)
label_y_train = label_encoder1.transform(Y_pl_)
label_y_test = label_encoder1.fit_transform(np.ravel(Y_pl_))

# 轉換 features, 使用 label encoder 將 Categorical 資料轉成 numerical 資料
label_x_train = X_pl_.apply(LabelEncoder().fit_transform)
label_x_test = test_X.apply(LabelEncoder().fit_transform)

labeled_origin_test_y = test_y.copy()
labeled_origin_test_y = LabelEncoder().fit_transform(np.ravel(test_y))


# 指定分類模型
prediction_model = AdaBoostClassifier(random_state=1, n_estimators=70,learning_rate=0.3) # GaussianNB()  KNeighborsClassifier() #AdaBoostClassifier(random_state=1, n_estimators=70,learning_rate=0.3)  #RandomForestClassifier(random_state = 1, max_depth = 18,n_estimators = 500, min_samples_split = 2, min_samples_leaf = 1) #XGBClassifier() #SGDClassifier(loss="log",penalty="l2", max_iter=400) #tree.DecisionTreeClassifier(max_depth=40, random_state=0) #RandomForestClassifier(max_depth=20, random_state=0) #tree.DecisionTreeClassifier()
prediction_model.fit(label_x_train, label_y_train)


pred = prediction_model.predict(label_x_test)

print("origin values:\n", test_y[0:5])
print("predicted values:", pred[0:5])

# 將驗證集與預測結果相比較,得出模型的好壞

# accuracy
#print(label_y_test)
#print(pred)
print("accuracy score:", accuracy_score(labeled_origin_test_y, pred))

# precise

print("precise score:",precision_score(labeled_origin_test_y, pred, average='weighted'))

# recall

print("recall score:", recall_score(labeled_origin_test_y, pred, average='weighted'))

# f1 score

print("f1 score in macro:",f1_score(labeled_origin_test_y, pred, average='macro'))

print("f1 score in micro:",f1_score(labeled_origin_test_y, pred, average='micro'))


print("\nmake predictions for new data:")


# 因為測試資料也會有 Categorical 資料型態出現, 所以也要做轉換

# 去掉d class 欄位
label_prediction_data = testing_data.drop(['Attrition'], axis=1).copy()

# 轉換 features, 使用 label encoder 將 Categorical 資料轉成 numerical 資料
label_prediction_data = testing_data.drop(['Attrition'], axis=1).apply(LabelEncoder().fit_transform)

result = prediction_model.predict(label_prediction_data)

result
transformed = []
for i in result:
    if i == 0:
        print("No")
    else:
        print("Yes")

origin values:
 786    Yes
422     No
890     No
320     No
280     No
Name: Attrition, dtype: object
predicted values: [1 0 1 0 1]
accuracy score: 0.5966101694915255
precise score: 0.7947003487146217
recall score: 0.5966101694915255
f1 score in macro: 0.5442388834793899
f1 score in micro: 0.5966101694915255

make predictions for new data:
Yes
Yes
No
Yes
Yes
No
No
Yes
Yes
No
No
No
Yes
No
No
No
Yes
Yes
Yes
Yes
No
No
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
No
No
No
Yes
No
Yes
No
Yes
No
Yes
No
No
No
No
No
No
Yes
No
Yes
Yes
No
Yes
Yes
No
Yes
Yes
No
Yes
No
No
No
Yes
No
Yes
No
No
No
Yes
Yes
No
No
Yes
Yes
Yes
Yes
Yes
No
No
Yes
Yes
No
No
No
No
No
Yes
No
Yes
No
Yes
Yes
Yes
No
No
No
Yes
No
Yes
Yes
Yes
Yes
Yes
Yes
No
No
No
Yes
Yes
No
No
Yes
No
No
No
No
No
No
Yes
No
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
No
No
No
No
No
Yes
Yes
No
Yes
No
Yes
Yes
Yes
Yes
No
No
Yes
Yes
No
No
No
Yes
No
No
Yes
Yes
Yes
No
Yes
No
No
No
Yes
No
No
Yes
No
No
No
Yes
No
No
No
Yes
No
Yes
Yes
Yes
Yes
Yes
No
Yes
Yes
No
No
No
No
No
No
No
No
No
N